In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.preprocessing import StandardScaler
import missingno as msno  # For visualizing missing data
from scipy.stats import chi2_contingency

# Configure visualizations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
%matplotlib inline

# Display all columns
pd.set_option('display.max_columns', None)

# Set random seed for reproducibility
np.random.seed(42)

# Define the path to the dataset
file_path = "../data/raw/kenyan_loan_default_dataset.csv"

# Load the dataset
df = pd.read_csv(file_path)

# Display the first few rows to get a sense of the data
print(f"Dataset shape: {df.shape}")
df.head()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Get basic information about the dataset
print("Dataset Information:")
df.info()

# Get summary statistics for numerical columns
print("\nSummary Statistics for Numerical Columns:")
df.describe().T

In [ ]:
# Check for missing values in each column
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Create a DataFrame to display missing values and percentages
missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage (%)': missing_percentage
})

# Display only columns with missing values, sorted by percentage
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values('Percentage (%)', ascending=False)
print("Columns with Missing Values:")
missing_df

# Visualize missing values
plt.figure(figsize=(12, 6))
msno.matrix(df)
plt.title('Missing Value Matrix', fontsize=16)
plt.show()

plt.figure(figsize=(12, 6))
msno.bar(df)
plt.title('Missing Value Bar Chart', fontsize=16)
plt.show()

In [ ]:
# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Check for duplicate loan_ids (which should be unique)
duplicate_loan_ids = df['loan_id'].duplicated().sum()
print(f"Number of duplicate loan_ids: {duplicate_loan_ids}")

In [ ]:
# Make a copy of the original dataframe for cleaning
df_clean = df.copy()

# Handle missing values based on column type
# For numerical columns, we'll fill with median
numerical_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns
for col in numerical_cols:
    if df_clean[col].isnull().sum() > 0:
        median_value = df_clean[col].median()
        df_clean[col] = df_clean[col].fillna(median_value)
        print(f"Filled missing values in {col} with median: {median_value}")

# For categorical columns, we'll fill with mode
categorical_cols = df_clean.select_dtypes(include=['object', 'bool']).columns
for col in categorical_cols:
    if df_clean[col].isnull().sum() > 0:
        mode_value = df_clean[col].mode()[0]
        df_clean[col] = df_clean[col].fillna(mode_value)
        print(f"Filled missing values in {col} with mode: {mode_value}")

# Check for outliers in numerical columns
plt.figure(figsize=(15, 10))
df_clean.select_dtypes(include=['int64', 'float64']).boxplot()
plt.title('Boxplot for Numerical Features', fontsize=16)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Handle outliers for key numerical columns using IQR method
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Count outliers
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column].count()
    print(f"{column}: {outliers} outliers detected")
    
    # Cap the outliers instead of removing
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    
    return df

# Apply outlier handling to key numerical columns
numerical_columns_for_outlier_treatment = ['age', 'monthly_income_kes', 'loan_amount_kes', 
                                          'interest_rate', 'mobile_money_usage', 'days_late']

for col in numerical_columns_for_outlier_treatment:
    df_clean = handle_outliers(df_clean, col)

# Create binary/dummy variables for categorical features
df_clean = pd.get_dummies(df_clean, columns=['gender', 'marital_status', 'education_level', 
                                            'employment_status', 'repayment_frequency', 
                                            'loan_purpose', 'collateral_type'], 
                         drop_first=True)

# Convert group_loan to numeric if it's not already
if df_clean['group_loan'].dtype == 'object':
    df_clean['group_loan'] = df_clean['group_loan'].map({'Yes': 1, 'No': 0})

# Verify data cleaning results
print("\nDataset shape after cleaning:", df_clean.shape)
print("Missing values after cleaning:", df_clean.isnull().sum().sum())
df_clean.head()

In [ ]:
# Analyze the distribution of the target variable
plt.figure(figsize=(10, 6))
default_counts = df_clean['defaulted'].value_counts()
default_percentage = 100 * default_counts / len(df_clean)

ax = sns.countplot(x='defaulted', data=df_clean, palette=['#2ecc71', '#e74c3c'])
plt.title('Distribution of Loan Defaults', fontsize=16)
plt.xlabel('Default Status (0 = No Default, 1 = Default)')
plt.ylabel('Count')

# Add percentages to the bars
for i, count in enumerate(default_counts):
    percentage = default_percentage[i]
    plt.text(i, count + 100, f'{percentage:.1f}%', ha='center', fontsize=12)

plt.tight_layout()
plt.show()

print(f"Number of non-defaults (0): {default_counts[0]} ({default_percentage[0]:.1f}%)")
print(f"Number of defaults (1): {default_counts[1]} ({default_percentage[1]:.1f}%)")

In [ ]:
# Select key numerical variables for analysis
numerical_vars = ['age', 'monthly_income_kes', 'mobile_money_usage', 'loan_amount_kes', 
                 'loan_term_months', 'interest_rate', 'num_previous_loans', 
                 'num_defaults', 'days_late', 'repayment_progress']

# Create histograms for each numerical variable
plt.figure(figsize=(15, 15))
for i, var in enumerate(numerical_vars):
    plt.subplot(4, 3, i+1)
    sns.histplot(df_clean[var], kde=True)
    plt.title(f'Distribution of {var}')
    plt.tight_layout()
plt.show()

# Create boxplots for each numerical variable by default status
plt.figure(figsize=(15, 15))
for i, var in enumerate(numerical_vars):
    plt.subplot(4, 3, i+1)
    sns.boxplot(x='defaulted', y=var, data=df_clean)
    plt.title(f'{var} by Default Status')
    plt.tight_layout()
plt.show()

# Summary statistics by default status
print("Summary statistics for non-default loans (0):")
df_clean[df_clean['defaulted'] == 0][numerical_vars].describe().T

print("\nSummary statistics for default loans (1):")
df_clean[df_clean['defaulted'] == 1][numerical_vars].describe().T

In [ ]:
# Original categorical columns before one-hot encoding
original_categorical_cols = ['gender', 'marital_status', 'education_level', 'county', 
                           'employment_status', 'repayment_frequency', 
                           'loan_purpose', 'collateral_type', 'group_loan']

# For each original categorical column, analyze distribution and relationship with defaults
plt.figure(figsize=(20, 25))
subplot_count = 1

for col in original_categorical_cols:
    if col in df:  # Check if the column exists in the original dataframe
        # Distribution of categories
        plt.subplot(len(original_categorical_cols), 2, subplot_count)
        value_counts = df[col].value_counts()
        sns.barplot(x=value_counts.index, y=value_counts.values)
        plt.title(f'Distribution of {col}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        subplot_count += 1
        
        # Default rate by category
        plt.subplot(len(original_categorical_cols), 2, subplot_count)
        default_rate = df.groupby(col)['defaulted'].mean().sort_values(ascending=False)
        sns.barplot(x=default_rate.index, y=default_rate.values)
        plt.title(f'Default Rate by {col}')
        plt.ylabel('Default Rate')
        plt.xticks(rotation=45, ha='right')
        plt.axhline(df['defaulted'].mean(), color='red', linestyle='--', 
                    label=f'Overall Default Rate: {df["defaulted"].mean():.2f}')
        plt.legend()
        plt.tight_layout()
        subplot_count += 1

plt.tight_layout(pad=3.0)
plt.show()

In [ ]:
# Calculate correlation matrix for numerical variables
correlation_matrix = df_clean.select_dtypes(include=['int64', 'float64']).corr()

# Plot the correlation heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Matrix of Numerical Variables', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Identify variables with strong correlation with the target
default_correlation = correlation_matrix['defaulted'].sort_values(ascending=False)
print("Correlation with Default Status:")
print(default_correlation)

# Plot the top correlated features with default status
plt.figure(figsize=(12, 8))
default_correlation = default_correlation.drop('defaulted')  # Remove self-correlation
top_correlations = default_correlation.abs().sort_values(ascending=False).head(10)
sns.barplot(x=top_correlations.values, y=top_correlations.index)
plt.title('Top 10 Features Correlated with Default Status', fontsize=16)
plt.xlabel('Correlation Coefficient')
plt.tight_layout()
plt.show()

In [ ]:
# Analyze key relationships between important variables

# 1. Income vs. Loan Amount by Default Status
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_clean, x='monthly_income_kes', y='loan_amount_kes', 
                hue='defaulted', palette=['#2ecc71', '#e74c3c'], alpha=0.7)
plt.title('Loan Amount vs. Monthly Income by Default Status', fontsize=16)
plt.xlabel('Monthly Income (KES)')
plt.ylabel('Loan Amount (KES)')
plt.legend(title='Default Status', labels=['No Default', 'Default'])
plt.tight_layout()
plt.show()

# 2. Loan-to-Income Ratio Analysis
df_clean['loan_to_income_ratio'] = df_clean['loan_amount_kes'] / df_clean['monthly_income_kes']

plt.figure(figsize=(12, 8))
sns.boxplot(x='defaulted', y='loan_to_income_ratio', data=df_clean)
plt.title('Loan-to-Income Ratio by Default Status', fontsize=16)
plt.xlabel('Default Status')
plt.ylabel('Loan-to-Income Ratio')
plt.tight_layout()
plt.show()

# 3. Default rate by Age Group
df_clean['age_group'] = pd.cut(df_clean['age'], bins=[18, 25, 35, 45, 55, 65, 100], 
                             labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+'])

plt.figure(figsize=(12, 8))
age_default_rate = df_clean.groupby('age_group')['defaulted'].mean().sort_values()
sns.barplot(x=age_default_rate.index, y=age_default_rate.values)
plt.title('Default Rate by Age Group', fontsize=16)
plt.xlabel('Age Group')
plt.ylabel('Default Rate')
plt.axhline(df_clean['defaulted'].mean(), color='red', linestyle='--', 
            label=f'Overall Default Rate: {df_clean["defaulted"].mean():.2f}')
plt.legend()
plt.tight_layout()
plt.show()

# 4. Default rate by Interest Rate Group
df_clean['interest_rate_group'] = pd.cut(df_clean['interest_rate'], bins=[0, 5, 10, 15, 20, 25, 30, 100], 
                                      labels=['0-5%', '5-10%', '10-15%', '15-20%', '20-25%', '25-30%', '30%+'])

plt.figure(figsize=(12, 8))
interest_default_rate = df_clean.groupby('interest_rate_group')['defaulted'].mean()
sns.barplot(x=interest_default_rate.index, y=interest_default_rate.values)
plt.title('Default Rate by Interest Rate', fontsize=16)
plt.xlabel('Interest Rate Range')
plt.ylabel('Default Rate')
plt.axhline(df_clean['defaulted'].mean(), color='red', linestyle='--', 
            label=f'Overall Default Rate: {df_clean["defaulted"].mean():.2f}')
plt.legend()
plt.tight_layout()
plt.show()

# 5. Default rate by Previous Defaults
plt.figure(figsize=(12, 8))
default_by_prev_defaults = df_clean.groupby('num_defaults')['defaulted'].mean()
sns.barplot(x=default_by_prev_defaults.index, y=default_by_prev_defaults.values)
plt.title('Default Rate by Number of Previous Defaults', fontsize=16)
plt.xlabel('Number of Previous Defaults')
plt.ylabel('Default Rate')
plt.axhline(df_clean['defaulted'].mean(), color='red', linestyle='--', 
            label=f'Overall Default Rate: {df_clean["defaulted"].mean():.2f}')
plt.legend()
plt.tight_layout()
plt.show()

# 6. Loan Amount by Purpose and Default Status
plt.figure(figsize=(15, 8))
purpose_cols = [col for col in df_clean.columns if 'loan_purpose_' in col]
purpose_data = pd.melt(df_clean, 
                       id_vars=['defaulted', 'loan_amount_kes'], 
                       value_vars=purpose_cols, 
                       var_name='loan_purpose', 
                       value_name='has_purpose')
purpose_data = purpose_data[purpose_data['has_purpose'] == 1]
purpose_data['loan_purpose'] = purpose_data['loan_purpose'].str.replace('loan_purpose_', '')

sns.boxplot(x='loan_purpose', y='loan_amount_kes', hue='defaulted', data=purpose_data)
plt.title('Loan Amount by Loan Purpose and Default Status', fontsize=16)
plt.xlabel('Loan Purpose')
plt.ylabel('Loan Amount (KES)')
plt.legend(title='Default Status')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze mobile money usage patterns
plt.figure(figsize=(12, 8))
sns.boxplot(x='defaulted', y='mobile_money_usage', data=df_clean)
plt.title('Mobile Money Usage by Default Status', fontsize=16)
plt.xlabel('Default Status')
plt.ylabel('Monthly M-Pesa Transaction Volume (KES)')
plt.tight_layout()
plt.show()

# Relationship between Mobile Money Usage, Loan Amount, and Default
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_clean, x='mobile_money_usage', y='loan_amount_kes', 
                hue='defaulted', size='monthly_income_kes', sizes=(20, 200), 
                palette=['#2ecc71', '#e74c3c'], alpha=0.7)
plt.title('Mobile Money Usage vs. Loan Amount by Default Status', fontsize=16)
plt.xlabel('Monthly M-Pesa Transaction Volume (KES)')
plt.ylabel('Loan Amount (KES)')
plt.legend(title='Default Status', labels=['No Default', 'Default'])
plt.tight_layout()
plt.show()

# Create a mobile money usage to income ratio
df_clean['mobile_money_to_income_ratio'] = df_clean['mobile_money_usage'] / df_clean['monthly_income_kes']

plt.figure(figsize=(12, 8))
sns.boxplot(x='defaulted', y='mobile_money_to_income_ratio', data=df_clean)
plt.title('Mobile Money Usage to Income Ratio by Default Status', fontsize=16)
plt.xlabel('Default Status')
plt.ylabel('Mobile Money Usage to Income Ratio')
plt.tight_layout()
plt.show()

# Mobile Money Usage by Loan Purpose
plt.figure(figsize=(15, 8))
purpose_cols = [col for col in df_clean.columns if 'loan_purpose_' in col]
purpose_data = pd.melt(df_clean, 
                       id_vars=['defaulted', 'mobile_money_usage'], 
                       value_vars=purpose_cols, 
                       var_name='loan_purpose', 
                       value_name='has_purpose')
purpose_data = purpose_data[purpose_data['has_purpose'] == 1]
purpose_data['loan_purpose'] = purpose_data['loan_purpose'].str.replace('loan_purpose_', '')

sns.boxplot(x='loan_purpose', y='mobile_money_usage', hue='defaulted', data=purpose_data)
plt.title('Mobile Money Usage by Loan Purpose and Default Status', fontsize=16)
plt.xlabel('Loan Purpose')
plt.ylabel('Monthly M-Pesa Transaction Volume (KES)')
plt.legend(title='Default Status')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze default rates by county
county_default_rates = df.groupby('county')['defaulted'].agg(['mean', 'count']).sort_values('mean', ascending=False)
county_default_rates.columns = ['Default Rate', 'Number of Loans']
county_default_rates = county_default_rates[county_default_rates['Number of Loans'] > 10]  # Filter out counties with few loans

plt.figure(figsize=(15, 8))
sns.barplot(x=county_default_rates.index, y=county_default_rates['Default Rate'])
plt.title('Loan Default Rate by County', fontsize=16)
plt.xlabel('County')
plt.ylabel('Default Rate')
plt.axhline(df['defaulted'].mean(), color='red', linestyle='--', 
            label=f'Overall Default Rate: {df["defaulted"].mean():.2f}')
plt.legend()
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Use a simple Random Forest model to estimate feature importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Prepare data for the model
X = df_clean.drop(['loan_id', 'customer_id', 'defaulted', 'county', 'age_group', 
                   'interest_rate_group'], axis=1)
y = df_clean['defaulted']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importances
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(12, 10))
sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
plt.title('Top 20 Feature Importance for Loan Default Prediction', fontsize=16)
plt.tight_layout()
plt.show()

print("Top 20 Most Important Features:")
print(feature_importance.head(20))